# Null Ellipsometry

**Scott Prahl**

*April 2020*

References

* Archer, *Manual on Ellipsometry* 1968.
* Azzam, *Ellipsometry and Polarized Light*, 1977.
* Fujiwara, *Spectroscopic Ellipsometry*, 2007.
* McCrackin, "Measurement of the Thickness and Refractive Index of Very Thin Films and the Optical Properties of Surfaces by Ellipsometry," 1963.
* Tompkins, *A User's Guide to Ellipsometry*, 1993
* Tompkins, *Handbook of Ellipsometry*, 2005.
* Woollam, *A short course in ellipsometry*, 2001.

In [1]:
import sympy
import numpy as np
import matplotlib.pyplot as plt

import pypolar.fresnel as fresnel
import pypolar.jones as jones
import pypolar.sym_jones as sym_jones
import pypolar.visualization as vis
import pypolar.ellipsometry as ellipso

sympy.init_printing(use_unicode=True)

## Null Ellipsometry

### Layout

<img src="pca.png" width="100%">

A typical layout for a null ellipsometer is shown below.  An unpolarized laser is incident on a linear polarizer set at an angle $\theta_p$.  The light then passes through a 
quarter wave plate (QWP) set at ±45° to the plane of incidence.  The light emerging from the polarizer will in general be elliptically polarized.  This light hits the sample and the reflected light passes through an analyzer (at $\theta_a$) before reaching the detector.

In null ellipsometry, the polarizer and analyzer are rotated until no light reaches the detector.  These angles are used to determine $\tan\psi$ and $\Delta$, which in turn, are used to calculate the complex refractive index of the sample.

### Zones

McCrackin summarizes

>For any given surface there is a multiplicity of
polarizer, analyzer, and compensator scale settings
that produce extinction by the analyzer of the
reflected light from the surface, and it becomes
somewhat of a problem to determine the values of
$\Delta$ and $\psi$ from these various readings. In order to
explain how these numerous readings arise and how
$\Delta$ and $\psi$ may be computed from them it is well to
keep two facts in mind: (a) all azimuthal angles are
measured positive counter-clockwise from the plane
of incidence when looking into the light beam, and
(b) the compensator, which may be set at any azimuth,
is generally set so that its fast axis is in an
azimuth of $\pm\pi/4$...

>The various readings fall into four sets called
zones, two with the fast axis of the compensator set
at $\pi/4$, numbered 2 and 4, and two with it set at
$-\pi/4$, numbered 1 and 3. In each zone there is
one independent set of polarizer and analyzer readings,
making four independent sets of $\theta_p$ and $\theta_a$
readings in all. However, since both analyzer and
polarizer may be rotated by $\pi$ without affecting the
results, there are 16 polarizer and analyzer settings
falling into four independent zones. Since the
compensator may also be rotated by $\pi$ without
affecting the results, there are 32 possible sets of
readings on the ellipsometer.


The various null angles fall into four sets called zones, two with the fast-axis
of the quarter wave plate set 45° (2 & 4) and two with the fast-axis of the
quarter wave plate set to -45° (1 & 3).  In each zone there are four combinations
of polarizer and analyzer angles that have a null reading (because rotation of a
linear polarizer by 180° should give the same result.  Identifying the right values to 
determine $\psi$ and $\Delta$ is confusing so
McCrackin introduces three new parameters ($p$, $a_s$, and $a_p$) to simplify
    
>Rather than calculating $\Delta$ and $\psi$ directly from the
$\theta_p$ and $\theta_a$ values, it is useful to calculate three other
quantities, $p$, $a_p$, and $a_s$, from the $\theta_p$ and $\theta_a$ values,
$p$ being related to the $\theta_p$ readings, $a_p$, related to the
$\theta_a$ readings in zones 1 and 4, and $a_s$, related to the
$\theta_a$ readings in zones 2 and 3. For a perfect quarterwave
plate these are related to $\Delta$ and $\psi$ by

$$
\Delta = \frac{\pi}{2} +2p
\qquad\mbox{and}\qquad
\psi=a_s = a_p
$$

| Zone  |  QWP Orientation  | Polarizer Angle $\theta_p$  |  Analyzer Angle $\theta_p$   |
|:-----:|:-------------:|:----------|:--------|
| 1     |    $-\pi/4$      | $p$        | $a_p$      |
|       |                  | $p+\pi$    | $a_p$      |
|       |                  | $p$        | $a_p+\pi$  |
|       |                  | $p+\pi$    | $a_p+\pi$  |
|       |                  |            |            |
| 3     |    $-\pi/4$      | $p+ \pi/2$ | $ \pi-a_s$ |
|       |                  | $p+3\pi/2$ | $ \pi-a_s$ |
|       |                  | $p+ \pi/2$ | $2\pi-a_s$ |
|       |                  | $p+3\pi/2$ | $2\pi-a_s$ |
|       |                  |            |            |
| 2     |    $+\pi/4$      | $ \pi/2-p$ | $a_s$      |
|       |                  | $3\pi/2-p$ | $a_s$      |
|       |                  | $ \pi/2-p$ | $a_s+\pi$  |
|       |                  | $3\pi/2-p$ | $a_s+\pi$  |
|       |                  |            |            |
| 4     |    $+\pi/4$      | $ \pi-p$   | $ \pi-a_p$ |
|       |                  | $2\pi-p$   | $ \pi-a_p$ |
|       |                  | $ \pi-p$   | $2\pi-a_p$ |
|       |                  | $2\pi-p$   | $2\pi-a_p$ |

So for an ellipsometer with the QWP at 45°, there are eight different polarizer/analyzer pairs that will null the reflected light.  The $\psi$ and $\Delta$ values are readily calculated from these using:

$$
\Delta = \frac{\pi}{2} +2p
\qquad\mbox{and}\qquad
\psi=a_s = a_p
$$

#### Zone 2

We know that $0\le\psi\le\pi/2$ and that $0\le\Delta\le2\pi$.  Therefore with a bit of algebra, the first row of zone 2 leads to the requirement that the analyzer must be between 0° and 90°:

$$
0°<\theta_{a2}<90° \quad\mbox{and}\quad -45°\le\theta_{p2}\le135°
$$

when **both** conditions are satisfied, then

$$
\psi = \theta_{a2} \quad\textit{and}\quad \Delta = 270°-2\theta_{p2} 
$$

#### Zone 4

For zone 4, the last row in Table 1 requires the analyzer to be between -90° and 0°

$$
-90°<\theta_{a4}<0° \quad\mbox{and}\quad -135°<\theta_{p4}<45°
$$

when **both** conditions are satisfied, then

$$
\psi = -\theta_{a4} \quad\textit{and}\quad \Delta = 90°-2\theta_{p4}
$$

Measurements in either zone should lead to the same value for $\psi$ and $\Delta$.  The average of the values from zones 2 and 4 should eliminate errors due to inexact compensation by the QWP.  

### BK7

In [2]:
np.pi/2

In [3]:
theta_i = np.radians(70)
m = 1.525

s = ellipso.null_angles_report(m, theta_i)
print(s)

m       = 1.5250+0.0000j
theta_i =    70.0°

zone  theta_p   theta_a
  1    315.0°     20.1°
  1    135.0°     20.1°
  1    315.0°    200.1°
  1    135.0°    180.0°

  3     45.0°    159.9°
  3    225.0°    159.9°
  3     45.0°    339.9°
  3    225.0°    339.9°

  2    135.0°     20.1°
  2    315.0°     20.1°
  2    135.0°    200.1°
  2    315.0°    200.1°

  4    225.0°    159.9°
  4     45.0°    159.9°
  4    225.0°    339.9°
  4     45.0°    339.9°

p       =   -45.0°
a       =    20.1°

psi     =    20.1°
Delta   =     0.0°




In [17]:
#### 
theta_i = np.radians(50)
print(np.degrees(theta_i))
P2 = np.radians(45)
A2 = np.radians(9.5)
rho2 = ellipso.rho_from_zone_2_null_angles(P2,A2)
m = ellipso.m_from_rho(rho2,theta_i)
print(m)

P4 = np.radians(315.0-360)
A4 = np.radians(348.7-360)
rho4 = ellipso.rho_from_zone_4_null_angles(P4,A4)
m = ellipso.m_from_rho(rho4,theta_i)
print(m)

theta_i = np.radians(70)
print(np.degrees(theta_i))
P2 = np.radians(135)
A2 = np.radians(19)
rho2 = ellipso.rho_from_zone_2_null_angles(P2,A2)
m = ellipso.m_from_rho(rho2,theta_i)
print(m)

P4 = np.radians(45)
A4 = np.radians(-20.1)
rho4 = ellipso.rho_from_zone_4_null_angles(P4,A4)
m = ellipso.m_from_rho(rho4,theta_i)
print(m)

50.0
(1.4916235768488333-4.630931219065213e-17j)
(1.568656820688237-6.089541259265148e-17j)
70.0
(1.5711963698409195+0j)
(1.5229061519292106+0j)


In [5]:
theta_i = np.radians(50)
m = 1.525

s = ellipso.null_angles_report(m, theta_i)
print(s)

m       = 1.5250+0.0000j
theta_i =    50.0°

zone  theta_p   theta_a
  1     45.0°     10.3°
  1    225.0°     10.3°
  1     45.0°    190.3°
  1    225.0°    180.0°

  3    135.0°    169.7°
  3    315.0°    169.7°
  3    135.0°    349.7°
  3    315.0°    349.7°

  2     45.0°     10.3°
  2    225.0°     10.3°
  2     45.0°    190.3°
  2    225.0°    190.3°

  4    135.0°    169.7°
  4    315.0°    169.7°
  4    135.0°    349.7°
  4    315.0°    349.7°

p       =    45.0°
a       =    10.3°

psi     =    10.3°
Delta   =   180.0°




### Amorphous Silicon

In [6]:
theta_i = np.radians(70)
m = 4.516 - 0.249j 
s = ellipso.null_angles_report(m, theta_i)
print("****Amorphous Silicon****\n\n",s)

****Amorphous Silicon****

 m       = 4.5160-0.2490j
theta_i =    70.0°

zone  theta_p   theta_a
  1     42.1°     14.8°
  1    222.1°     14.8°
  1     42.1°    194.8°
  1    222.1°    180.0°

  3    132.1°    165.2°
  3    312.1°    165.2°
  3    132.1°    345.2°
  3    312.1°    345.2°

  2     47.9°     14.8°
  2    227.9°     14.8°
  2     47.9°    194.8°
  2    227.9°    194.8°

  4    137.9°    165.2°
  4    317.9°    165.2°
  4    137.9°    345.2°
  4    317.9°    345.2°

p       =    42.1°
a       =    14.8°

psi     =    14.8°
Delta   =   174.2°




### Crystalline Silicon

In [7]:
theta_i = np.radians(70)
m = 3.875 - 0.023j  # crystalline silicon
s = ellipso.null_angles_report(m, theta_i)
print("****Crystalline Silicon****\n\n",s)

****Crystalline Silicon****

 m       = 3.8750-0.0230j
theta_i =    70.0°

zone  theta_p   theta_a
  1     44.5°     10.5°
  1    224.5°     10.5°
  1     44.5°    190.5°
  1    224.5°    180.0°

  3    134.5°    169.5°
  3    314.5°    169.5°
  3    134.5°    349.5°
  3    314.5°    349.5°

  2     45.5°     10.5°
  2    225.5°     10.5°
  2     45.5°    190.5°
  2    225.5°    190.5°

  4    135.5°    169.5°
  4    315.5°    169.5°
  4    135.5°    349.5°
  4    315.5°    349.5°

p       =    44.5°
a       =    10.5°

psi     =    10.5°
Delta   =   179.1°




In [18]:
theta_i = np.radians(50)
print(np.degrees(theta_i))

P2 = np.radians(42.3)
A2 = np.radians(32.1)

rho2 = ellipso.rho_from_zone_2_null_angles(P2,A2)
m = ellipso.m_from_rho(rho2,theta_i)
print(m)

P4 = np.radians(316.5-360)
A4 = np.radians(328.5-360)
rho4 = ellipso.rho_from_zone_4_null_angles(P4,A4)
m = ellipso.m_from_rho(rho4,theta_i)
print(m)

theta_i = np.radians(70)
print(np.degrees(theta_i))


P2 = np.radians(55.8)
A2 = np.radians(10.4)

rho2 = ellipso.rho_from_zone_2_null_angles(P2,A2)
m = ellipso.m_from_rho(rho2,theta_i)
print(m)

P4 = np.radians(-34.2)
A4 = np.radians(-10.9)
rho4 = ellipso.rho_from_zone_4_null_angles(P4,A4)
m = ellipso.m_from_rho(rho4,theta_i)
print(m)

50.0
(3.905633098928391+0.7320990564489631j)
(3.8369866335581917-0.3785649340861991j)
70.0
(3.7215641030895554-0.48783161452493856j)
(3.777718069991995-0.5224990856172298j)


In [ ]:
theta = sympy.Symbol('theta', real=True)
m = sympy.Symbol('m', complex=True)

theta

In [ ]:
QWP = sym_jones.op_quarter_wave_plate(sympy.numbers.pi/4)
QWP

In [ ]:
POL = 1/sympy.sqrt(2)*sym_jones.field_linear(theta)
POL

In [ ]:
help(sym_jones.field_elliptical)

In [ ]:
delta_ip = sympy.Symbol('delta_ip', real=True)
delta_is = sympy.Symbol('delta_is', real=True)

Eip = sympy.Symbol('E_ip', real=True)
Eis = sympy.Symbol('E_is', real=True)

Ei = sympy.Matrix([Eip*sympy.exp(delta_ip*sympy.I),Eip*sympy.exp(delta_is*sympy.I)])
Ei

In [ ]:
Er = REFL * Ei

Er

In [ ]:
REFL = sym_jones.op_fresnel_reflection(m, theta)
sympy.simplify(REFL[0,0]/REFL[1,1])

In [ ]:
dir(sym_jones)

In [ ]:
r_p = sympy.Symbol('r_p', complex=True)
r_s = sympy.Symbol('r_s', complex=True)
theta_p = sympy.Symbol('theta_p', complex=True)
theta_a = sympy.Symbol('theta_a', complex=True)
E0 = sympy.Symbol('E_0', real=True)

J = E0 * sym_jones.field_horizontal()

rotate1 = sym_jones.op_rotation(-theta_p)

refl = sympy.Matrix([[r_p,0],[0,r_s]])

rotate2 = sym_jones.op_rotation(theta_a)

analyzer = sym_jones.op_linear_polarizer(0)

analyzer * rotate2 * refl * rotate1 * J

In [ ]:
intensity = sympy.simplify(sym_jones.intensity(QWP * POL))
intensity

In [ ]:
phase = sympy.simplify(sym_jones.phase(QWP * POL))
phase

In [ ]:
ph = sympy.lambdify((theta), phase)
ph(0.1)

In [ ]:
th = np.linspace(0,np.pi,100)
plt.plot(np.degrees(th),np.degrees(ph(th)))
plt.yticks([90,0,-90,-180,-270])
plt.xticks([0,45,90,135,180])
plt.grid(True)
plt.ylabel('Polarization Phase of Exiting Light')
plt.xlabel('Linear Polarization Angle')
plt.title('Linearly polarized light passing through a QWP')
plt.show()

In [ ]:
qwp = jones.op_quarter_wave_plate(np.pi/4)

plt.subplots(4,4,figsize=(8,8))
for i in range(16):
    theta = np.radians(15*i)
    pol = jones.field_linear(theta)
    result = qwp @ pol
    vis.draw_jones_ellipse(result)
        
    

In [ ]:
angle = np.linspace(0,90,100)

m = 1.5

rp = fresnel.r_par(m,np.radians(angle))
rs = fresnel.r_per(m,np.radians(angle))

plt.plot(angle,rp,label='$r_p$')
plt.plot(angle,rs,label='$r_s$')
plt.axhline(0,color='black')
plt.axvline(50,color='black')
plt.axvline(70,color='black')


plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Field Reflectance')
plt.title('Glass')
plt.legend()
plt.show()

In [ ]:
plt.plot(angle,np.degrees(np.arctan(np.abs(rp/rs))),label='arctan($r_p/r_s$)')
#plt.axhline(0,color='black')
plt.axvline(50,color='black')
plt.axvline(70,color='black')

plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Field Reflectance')
plt.title('Glass')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.plot(angle,np.angle(rs)-np.angle(rp),label='$r_p-r_s$')
plt.axvline(50,color='black')
plt.axvline(70,color='black')

plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Phase of Reflectance')
plt.title('Glass')
plt.legend()
plt.show()

In [ ]:
angle = np.linspace(0,90,100)

m = 3.88 - 0.15j

rp = fresnel.r_par(m,np.radians(angle))
rs = fresnel.r_per(m,np.radians(angle))

plt.plot(angle,rp,label='$r_p$')
plt.plot(angle,rs,label='$r_s$')
plt.axhline(0,color='black')
plt.axvline(50,color='black')
plt.axvline(70,color='black')

plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Field Reflectance')
plt.title('Silicon')
plt.legend()
plt.show()

In [ ]:
plt.plot(angle,np.degrees(np.arctan(np.abs(rp/rs))),label='arctan($r_p/r_s$)')
#plt.axhline(0,color='black')
plt.axvline(50,color='black')
plt.axvline(70,color='black')

plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Field Reflectance')
plt.title('Silicon')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.plot(angle,np.angle(rp),label='$r_p$')
plt.plot(angle,np.angle(rs),label='$r_s$')
plt.axvline(50,color='black')
plt.axvline(70,color='black')

plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Phase of Reflectance')
plt.title('Silicon')
plt.legend()
plt.show()

In [ ]:
plt.plot(angle,np.degrees(np.angle(rs)-np.angle(rp)),label='$r_p-r_s$')
plt.axvline(50,color='black')
plt.axvline(70,color='black')
plt.axhline(180,color='black',linestyle=':')
plt.xlabel('Angle of Incidence (°)')
plt.ylabel('Phase Difference')
plt.title('Silicon')
plt.legend()
plt.show()

In [ ]:
def test_index(P1,P2,A1,A2,thetai):
    psi, Delta = null_experiment(np.radians(P1),np.radians(P2),np.radians(A1),np.radians(A2))
    m = ellipsometry_index2(Delta,np.tan(psi),np.radians(thetai))
    return m

thetai = 70
A1 = 20
P1 = 135
A2 = 155
P2 = 230

m = test_index(P1,P2,A1,A2,thetai)

error = 2

m1 = test_index(P1+error,P2,A1,A2,thetai)
m2 = test_index(P1,P2+error,A1,A2,thetai)
m3 = test_index(P1,P2,A1+error,A2,thetai)
m4 = test_index(P1,P2,A1,A2+error,thetai)

print("% .4f"%np.real(m))
print("% .4f"%np.real(m-m1))
print("% .4f"%np.real(m-m2))
print("% .4f"%np.real(m-m3))
print("% .4f"%np.real(m-m4))


Now validate that the known index of refraction is recovered and that the equation above is valid.

In [ ]:
# Now calculate the 
m2 = fresnel.ellipsometry_index(rho, theta)

plt.plot(th, m2.real)
plt.plot(th, -m2.imag)

plt.annotate(r'n', xy=(th[-1],m2[-1].real), color='blue')
plt.annotate(r'$\kappa$', xy=(th[-1],-m2[-1].imag), color='red')

plt.xlabel(r"$\theta$ (degrees)")
plt.ylabel('Real or Imaginary part of Refractive Index')
plt.title('Refractive Index = %.4f %.4fj'%(m.real,m.imag))

plt.show()

Quantitatively show that things work out

In [ ]:
th = 70
theta = th*np.pi/180
m = 1.5 - 2.0j
print('At and incidence angle of %.1f'%th)
print('Refractive   Index = {:.3f}'.format(m))

rho = fresnel.ellipsometry_rho(m,theta)
m2 = fresnel.ellipsometry_index(rho,theta)         
print('Ellipsometry Index = {:.3f} using rho'.format(m2))